In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms

from data.pipeline import get_data_raw

In [ ]:
from torch import nn, tanh
from torch.nn import Linear, Conv2d, ConvTranspose2d, BatchNorm2d, ReLU, LeakyReLU

In [ ]:
input_dim = 32
num_channels = 3
hidden_dim = 16

In [ ]:
inp = torch.randn(4, num_channels, input_dim, input_dim)

In [ ]:
conv1 = Conv2d(in_channels=num_channels, out_channels=64, kernel_size=5, stride=2, padding=2)

In [ ]:
torch.std(conv1(inp))

In [ ]:
conv1(inp).shape

In [ ]:
conv1_bn = BatchNorm2d(64)

In [ ]:
torch.mean(conv1_bn(conv1(inp)))

In [ ]:
conv2 = Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2)

In [ ]:
conv2(conv1(inp)).shape

In [ ]:
conv3 = Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=2, padding=2)

In [ ]:
conv3(conv2(conv1(inp))).shape

In [ ]:
conv4 = Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=2, padding=2)

In [ ]:
conv4(conv3(conv2(conv1(inp)))).shape

In [ ]:
that_size = conv4(conv3(conv2(conv1(inp)))).shape

In [ ]:
fc1 = Linear(2048, 16)

In [ ]:
fc1(conv4(conv3(conv2(conv1(inp)))).view((4, -1))).shape

In [ ]:
class CustomView(nn.Module):  # Flattening layer for nn.Sequential
    def __init__(self, shape):
        super(CustomView, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)

In [ ]:
encoder = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, out_channels=64, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(),
            nn.BatchNorm2d(512),
            CustomView((-1, 2048)),
            nn.Linear(2048, hidden_dim)
        )

In [ ]:
encoder(inp)

In [ ]:
decoder = nn.Sequential(
            nn.Linear(hidden_dim, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            CustomView((-1, 512, 2, 2)),
            nn.ConvTranspose2d(512, 256, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=0 if input_dim==28 else 1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.ConvTranspose2d(64, num_channels, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
)

In [ ]:
decoder(encoder(inp)).shape#.permute((0, 2, 3, 1))

In [ ]:
means = (1, 2, 3)

In [ ]:
np.expand_dims(means, axis=[1, 2]).shape

In [ ]:
means = np.expand_dims(means, axis=[1, 2])

In [ ]:
decoder(encoder(inp)).detach().numpy() * means

In [ ]:
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

In [ ]:
parser = ArgumentParser()
parser.add_argument('--batch_size', default=64, type=int)
parser.add_argument('--hidden_dim', default=16, type=int)
parser.add_argument('--dataset', default='cifar', type=str)
parser.add_argument('--num_classes', default=10, type=int)
parser.add_argument('--party_data_size', default=4000, type=int)
parser.add_argument('--candidate_data_size', default=10000, type=int)
parser.add_argument('--split', default='equaldisjoint', type=str)
parser.add_argument('-f', type=str)
args = parser.parse_args()

In [ ]:
party_datasets, party_labels, candidate_dataset, candidate_labels = get_data_raw(dataset=args.dataset,
                                                                                         num_classes=args.num_classes,
                                                                                         party_data_size=args.party_data_size,
                                                                                         candidate_data_size=args.candidate_data_size,
                                                                                         split=args.split)

In [ ]:
party = 0

In [ ]:
plt.imshow(merge(party_datasets[party][:64], [8,8]))

In [ ]:
party_labels[party][:64]

In [ ]:
plt.imshow(merge(candidate_dataset[:64], [8,8]))

In [ ]:
candidate_labels[:64]

In [ ]:
np.max(party_datasets)

In [ ]:
np.min(party_datasets)

In [ ]:
np.max(candidate_dataset)

In [ ]:
np.min(candidate_labels)

In [ ]:
num_channels = party_datasets.shape[-1]

In [ ]:
np.mean(candidate_dataset.reshape([-1, num_channels]), axis=0)

In [ ]:
combined = np.concatenate([np.concatenate(party_datasets), candidate_dataset])

In [ ]:
combined.shape

In [ ]:
means = np.mean(combined.reshape(-1, num_channels), axis=0)

In [ ]:
means

In [ ]:
stds = np.std(combined.reshape(-1, num_channels), axis=0)

In [ ]:
stds

In [ ]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)

In [ ]:
datasets = []
for i in range(len(party_datasets)):
    transformed = (party_datasets[i] - means) / stds
    dataset = TensorDataset(torch.tensor(transformed), torch.tensor(party_labels[i]))
    datasets.append(dataset)
transformed = (candidate_dataset - means) / stds
dataset = TensorDataset(torch.tensor(transformed), torch.tensor(candidate_labels))
datasets.append(dataset)

In [ ]:
concat_dataset = ConcatDataset(*datasets)

In [ ]:
loader = torch.utils.data.DataLoader(
            concat_dataset,
            batch_size=args.batch_size,
            shuffle=True,
            pin_memory=True)

In [ ]:
my_iterator = iter(loader)

In [ ]:
party = 5

In [ ]:
batch_images, batch_labels = next(my_iterator)[party]

In [ ]:
plt.imshow(merge(batch_images, [8, 8]))

In [ ]:
batch_labels